In [59]:
import requests
import polyline
import pandas as pd
import math
from numpy import cos, sin, arcsin, sqrt, arctan2, radians

def onSegment (start, end, x, y):
    x1 = start[0]
    y1 = start[1]
    x2 = end[0]
    y2 = end[1]
    dist = abs((y2-y1)*x - (x2-x1)*y + x2*y1 - y2*x1)/math.sqrt((y2-y1)**2 + (x2-x1)**2)
    if dist < 0.0003 and x <= max(x1,x2) and x >= min(x1,x2) and y <= max(y1,y2) and y >= min(y1,y2):
        return True
    return False

def getRoute(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat):
    
    loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
    url = "http://router.project-osrm.org/route/v1/driving/"
    r = requests.get(url + loc + "?alternatives=true")
    print(url + loc + "?alternatives=true")
    if r.status_code!= 200:
        return {}
  
    res = r.json()   
    options = len(res['routes'])
    routes = []
    distances = []
    for i in range(options):
        routes.append(polyline.decode(res['routes'][i]['geometry']))
        distances.append(res['routes'][i]['distance'])
    start_point = [res['waypoints'][0]['location'][1], res['waypoints'][0]['location'][0]]
    end_point = [res['waypoints'][1]['location'][1], res['waypoints'][1]['location'][0]]
    
    out = { 'options': options,
            'route':routes,
            'start_point':start_point,
            'end_point':end_point,
            'distance':distances
          }

    return out

def getPoints(pickup, dropoff, df):
    route = getRoute(pickup[1], pickup[0], dropoff[1], dropoff[0])
    final_df = pd.DataFrame(columns=['date time', 'longitude', 'latitude', 'label', 'segment'])

    segments = []

    for i in range(len(route['route'][0]) - 1):
        #print(route['route'][0][i][1], route['route'][0][i][0])
        start = (route['route'][0][i][0], route['route'][0][i][1])
        end = (route['route'][0][i+1][0], route['route'][0][i+1][1])

        df3 = df[df.apply(lambda x: onSegment(start, end, x['latitude'], x['longitude']), axis=1)]
        df3['segment'] = i

        segments.append({
            'index': i,
            'start': start,
            'end': end
        })
        
        final_df = pd.concat([final_df, df3])

    return final_df, route['route']

def getColors(df):
    return 0





In [60]:
import folium

def plotRoute(route, df):
    # dictionary for colors
    colorsPoint = {0:'green', 1:'orange', 2:'red', 3:'orange', 4:'green'}

    # create a map
    # folium map dark theme
    m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=12,tiles='cartodbpositron')

    # plot the location points with color indicating the label
    for i in range(0, len(df)):
        folium.CircleMarker([df.iloc[i]['latitude'], df.iloc[i]['longitude']],
                            radius=0.2,
                            color=colorsPoint[df.iloc[i]['label']],
                            fill=True,
                            fill_color=colorsPoint[df.iloc[i]['label']],
                            fill_opacity=0.1).add_to(m)
# plot the route
    colorsRoute = ['blue', 'white', 'black', 'yellow', 'orange', 'purple', 'pink', 'black', 'white', 'gray']
    for i in range(0,len(route)):
        for j in range(0,len(route[i])):
            folium.Circle(
                location=[route[i][j][0], route[i][j][1]],
                radius=1,
                color=colorsRoute[i],
                fill=True,
                fill_color='crimson'
            ).add_to(m)
            #draw lines between points
            if j < len(route[i])-1:
                folium.PolyLine(locations=[[route[i][j][0], route[i][j][1]], [route[i][j+1][0], route[i][j+1][1]]], color=colorsRoute[i], weight=2.5, opacity=1).add_to(m)

    return m

choice = int(input("Enter the scenario number: "))
if choice == 1:
    # Tiananmen to Hilton Beijing Hotem
    pickup = (39.90772518863834, 116.39751663173872)
    dropoff = (39.95380284673872, 116.46232507838539)
elif choice == 2:
    # Temple of Sun to Beijing Capital International Airport
    pickup = (39.913353949958264, 116.44391608840026)
    dropoff = (40.085754497062055, 116.6048974654586)
elif choice == 3:
    # Beijing Zoo to Wanning Bridge
    pickup = (39.93983489654245, 116.34012668521959)
    dropoff = (39.93657649154817, 116.39602849390053)
elif choice == 4:
    pickup = (39.86727466612916, 116.31253609321384)
    dropoff = (39.84981380032512, 116.34614515651923)
elif choice == 5:
    # hihgway to highway
    pickup = (39.861812076670375, 116.45517167743945)
    dropoff = (39.842329091349136, 116.47901118418928)


df = pd.read_csv('data/hours/12Htest.txt', header=None, names=['date time', 'longitude', 'latitude', 'label'])
result, route = getPoints(pickup, dropoff, df)
m = plotRoute(route['route'], result)
m

http://router.project-osrm.org/route/v1/driving/116.39751663173872,39.90772518863834;116.46232507838539,39.95380284673872?alternatives=true


C:\Users\jules\AppData\Local\Temp\ipykernel_10944\562793832.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['segment'] = i
C:\Users\jules\AppData\Local\Temp\ipykernel_10944\562793832.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['segment'] = i
C:\Users\jules\AppData\Local\Temp\ipykernel_10944\562793832.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [62]:
# pip install Flask
from flask import Flask, request, jsonify
from datetime import datetime

app = Flask(__name__)

@app.route('/')
def hello_world():
   return {
         "message": "Hello World"
   }

# request example :
# http://127.0.0.1:5000/machine_learning?pickup_lat=39.90772518863834&pickup_lon=116.39751663173872&dropoff_lat=39.95380284673872&dropoff_lon=116.46232507838539
@app.route('/machine_learning', methods = ['GET'])
def machine_learning():
    args = request.args
    pickup_lat = float(args.get('pickup_lat'))
    pickup_lon = float(args.get('pickup_lon'))
    dropoff_lat = float(args.get('dropoff_lat'))
    dropoff_lon = float(args.get('dropoff_lon'))

    pickup = (float(pickup_lat), float(pickup_lon))
    dropoff = (float(dropoff_lat), float(dropoff_lon))

    hour = str(datetime.now().hour)

    if pickup_lon < 115.875363 or pickup_lon > 117.351154 or pickup_lat < 39.482463 or pickup_lat > 40.315493:
        return {
            "message": "The pickup location is out of bounds",
            "status": 400,
            "pickup_lat": pickup_lat,
            "pickup_lon": pickup_lon
        }
    elif dropoff_lon < 115.875363 or dropoff_lon > 117.351154 or dropoff_lat < 39.482463 or dropoff_lat > 40.315493:
        return {
            "message": "The dropoff location is out of bounds",
            "status": 400
        }
    elif dropoff_lat == pickup_lat and dropoff_lon == pickup_lon:
        return {
            "message": "The pickup and dropoff locations are the same",
            "status": 400
        }
    elif hour == None or pickup_lat == None or pickup_lon == None or dropoff_lat == None or dropoff_lon == None:
        return {
            "message": "Missing parameters",
            "status": 400
        }
    else:
        #df = pd.read_csv('data/hours/{}Htest.txt'.format(hour), header=None, names=['date time', 'longitude', 'latitude', 'label'])
        df = pd.read_csv('data/hours/12Htest.txt', header=None, names=['date time', 'longitude', 'latitude', 'label'])
        result, route = getPoints(pickup, dropoff, df)
        return {
            "pickup": pickup,
            "dropoff": dropoff,
            "hour": hour,
            "result": result.to_json(orient='records'),
            "route": route['route']
        }
    # 39.90772518863834 116.39751663173872 39.95380284673872 116.46232507838539

if __name__ == '__main__':
   app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


http://router.project-osrm.org/route/v1/driving/116.39751663173872,39.90772518863834;116.46232507838539,39.95380284673872?alternatives=true


C:\Users\jules\AppData\Local\Temp\ipykernel_10944\562793832.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['segment'] = i
C:\Users\jules\AppData\Local\Temp\ipykernel_10944\562793832.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['segment'] = i
C:\Users\jules\AppData\Local\Temp\ipykernel_10944\562793832.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta